Tech Challenge - FIAP - Fase 2 - Grupo 23

Bucket S3: techchallengefiapgrupo23

Microsoft Edge

In [1]:
import os
import time
import pandas as pd
import boto3
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

AWS_ACESS_KET_ID = os.environ.get('AWS_ACESS_KET_ID')
AWS_SECRET_ACESS_KEY = os.environ.get('AWS_SECRET_ACESS_KEY')

# Configuração do WebDriver (EdgeDriver)
edge_options = Options()
service = Service(r'C:\Users\AGFAKZZ\Downloads\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service, options=edge_options)

def download_file(driver, download_dir):
    # Abrir a página da web
    driver.get("https://arquivos.b3.com.br/Web/Consolidated")
    
    # Espera até que o botão de download esteja presente e clicável, e clica nele
    download_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div/div[2]/div[1]/div/div[1]/div[2]/div/div/div[1]/div[2]/p[2]/a[1]'))
    )
    download_button.click()
    
    # Espera o arquivo ser baixado
    time.sleep(20)  # Ajuste o tempo conforme necessário

    # Nome do arquivo baixado
    filename = 'TradeInformationConsolidatedAfterHoursFile_20240910_1.csv'
    file_path = os.path.join(download_dir, filename)
    
    if os.path.exists(file_path):
        return file_path
    else:
        raise FileNotFoundError(f"Arquivo não encontrado: {file_path}")

def convert_to_parquet(input_file_path, output_file_path):
    # Carregar o arquivo CSV em um DataFrame com opções robustas
    try:
        df = pd.read_csv(input_file_path, delimiter=';', on_bad_lines='warn')
    except pd.errors.ParserError as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return

    # Adicionar uma coluna de data se não existir
    if 'date' not in df.columns:
        df['date'] = datetime.now().strftime('%Y-%m-%d')
    
    # Salvar o DataFrame em formato Parquet
    try:
        df.to_parquet(output_file_path, index=False)
    except Exception as e:
        print(f"Erro ao salvar o arquivo Parquet: {e}")
        return

def upload_to_s3(file_path, bucket_name, s3_key):
    # Configurar o cliente S3 (as credenciais são lidas automaticamente do arquivo credentials)
    s3_client = boto3.client(
        's3'
    )
    
    print(f"Iniciando o upload do arquivo {file_path} para o bucket {bucket_name} com a chave {s3_key}.")
    
    try:
        s3_client.upload_file(file_path, bucket_name, s3_key)
        print("Upload concluído com sucesso.")
    except Exception as e:
        print(f"Erro ao fazer upload para o S3: {e}")

try:
    # Diretório de download
    download_dir = r'C:\Users\AGFAKZZ\Downloads'
    
    # Diretório para armazenar o arquivo Parquet
    bovespa_dir = r'C:\Users\AGFAKZZ\Desktop\FIAP\Fase 2 - Big Data Architecture\bovespa'
    
    # Crie o diretório bovespa se não existir
    os.makedirs(bovespa_dir, exist_ok=True)
    
    # Baixar o arquivo
    file_path = download_file(driver, download_dir)
    
    # Nome do arquivo Parquet a ser salvo no diretório bovespa
    parquet_file_path = os.path.join(bovespa_dir, 'arquivo.parquet')
    
    # Converter o arquivo para Parquet
    convert_to_parquet(file_path, parquet_file_path)
    
    # Upload para o S3
    bucket_name = 'techchallengefiapgrupo23'
    s3_key = 'arquivo.parquet'  # Ajuste o caminho conforme necessário
    upload_to_s3(parquet_file_path, bucket_name, s3_key)
    
    print("Arquivo processado e enviado para o S3 com sucesso.")

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # O navegador não será fechado automaticamente
    print("Navegador ainda aberto. Feche-o manualmente quando terminar.")

Iniciando o upload do arquivo C:\Users\AGFAKZZ\Desktop\FIAP\Fase 2 - Big Data Architecture\bovespa\arquivo.parquet para o bucket techchallengefiapgrupo23 com a chave arquivo.parquet.
Erro ao fazer upload para o S3: Failed to upload C:\Users\AGFAKZZ\Desktop\FIAP\Fase 2 - Big Data Architecture\bovespa\arquivo.parquet to techchallengefiapgrupo23/arquivo.parquet: An error occurred (InvalidToken) when calling the PutObject operation: The provided token is malformed or otherwise invalid.
Arquivo processado e enviado para o S3 com sucesso.
Navegador ainda aberto. Feche-o manualmente quando terminar.
